# AUDL AI Game Predictor

### Load Imports

The Nerual Network will be created using Keras and the data will be in the form of a pandas dataframe

In [1]:
#Imports

import tensorflow as tf
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout
from keras.regularizers import l2

### Reading the data

This cell reads the dataframe created in **Data_Constructor**. It will strip off the identifying information about each game (team names, dates) and store that in another dataframe. I do this becuase the data for these columns are not numerical and thus I do not want to train the model using these as inputs. Also, I do not want the model basing predictions based on a date, or a team naem. I want it to only base its predictions off team statistics. 

In [2]:
# Read the input data
computed_full = pd.read_csv(
    "AUDL_pd.csv",
    header = 0)

# Same as computed but without the identifying information
computed = computed_full.drop(columns=['Team Name','Abb', 'Opposing Team Name', 'Opponent Abb', 'Date'])
computed_labels = computed.pop("Result")


print(computed.shape)
computed.head()


(833, 64)


,Home Game,Game Number,Oppenent Game Number,Game Against Opponent,Completion% (Last 5 Games),Completion% (Season),Completion% Opponent (Last 5 Games),Completion% Opponent (Season),Completion% Against Opponent (Season),Completion% Opponent Against Home (Season),...,Blocks Opponent (Last 5 Games),Blocks Opponent (Season),Blocks Against Opponent (Season),Blocks Opponent Against Home (Season),Points/Total (Last 5 Games),Points/Total (Season),Points/Total Opponent (Last 5 Games),Points/Total Opponent (Season),Points/Total Against Opponent (Season),Points/Total Opponent Against Home (Season)
0,1,1,1,1,0.948,0.948,0.908,0.908,0.948,0.908,...,7.2,7.2,8.7,7.2,0.499,0.499,0.388,0.388,0.499,0.388
1,0,1,1,1,0.950,0.950,0.948,0.948,0.950,0.948,...,10.6,10.6,10.6,10.6,0.551,0.551,0.567,0.567,0.551,0.567
2,0,1,1,1,0.895,0.895,0.929,0.929,0.895,0.929,...,9.4,9.4,8.4,9.4,0.408,0.408,0.489,0.489,0.408,0.489
3,1,1,1,1,0.948,0.948,0.950,0.950,0.948,0.950,...,10.6,10.6,10.6,10.6,0.567,0.567,0.551,0.551,0.567,0.551
4,0,1,1,1,0.908,0.908,0.948,0.948,0.908,0.948,...,8.7,8.7,7.2,8.7,0.388,0.388,0.499,0.499,0.388,0.499


### Creating and training the model


The first cell spilts the pandas dataframe up into training and test data. It also designates which column the model will be trying to predict. In this case, it is whether the game resulted in a win or a loss.

In [3]:
# splits into training and test data
# test_size is the percent of data set aside for testing
x_train, x_test, y_train, y_test = train_test_split(computed, computed_labels, test_size=0.2)
# Converts to tensor object
x_train = tf.convert_to_tensor(x_train)
x_test = tf.convert_to_tensor(x_test)
y_train = tf.convert_to_tensor(y_train)
y_test = tf.convert_to_tensor(y_test)

In [8]:
# Creates Model
model = tf.keras.models.Sequential([
    Dense(64, activation='relu'),
    Dense(35,kernel_regularizer=l2(0.01), activation='relu'),
    #Dropout(.1),
    Dense(10,kernel_regularizer=l2(0.01), activation='relu'),
    Dense(1, kernel_regularizer=l2(0.01), activation='sigmoid')
    ])

# Compiles the model
model.compile(optimizer='sgd',
    loss='mean_squared_error',
    metrics=['accuracy'])

In [9]:
# Trains the model

batch_size = 8
epochs = 10


model.fit(x_train, y_train, batch_size,
    epochs,
    validation_data=(x_test, y_test))
# stores the predictions of the test data in a variable
predictions = model.predict(x_test)

# Evaluates model based on accuracy
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/10
84/84 [==============================] - 2s 6ms/step - loss: 1.1273 - accuracy: 0.4940 - val_loss: 1.0881 - val_accuracy: 0.5210
Epoch 2/10
84/84 [==============================] - 0s 2ms/step - loss: 0.8969 - accuracy: 0.5751 - val_loss: 0.8306 - val_accuracy: 0.5269
Epoch 3/10
84/84 [==============================] - 0s 3ms/step - loss: 0.7870 - accuracy: 0.6772 - val_loss: 0.7510 - val_accuracy: 0.7425
Epoch 4/10
84/84 [==============================] - 0s 3ms/step - loss: 0.7499 - accuracy: 0.7282 - val_loss: 0.7255 - val_accuracy: 0.7365
Epoch 5/10
84/84 [==============================] - 0s 3ms/step - loss: 0.7252 - accuracy: 0.7237 - val_loss: 0.7044 - val_accuracy: 0.7545
Epoch 6/10
84/84 [==============================] - 0s 3ms/step - loss: 0.7009 - accuracy: 0.7342 - val_loss: 0.7325 - val_accuracy: 0.6766
Epoch 7/10
84/84 [==============================] - 0s 3ms/step - loss: 0.6873 - accuracy: 0.7447 - val_loss: 0.6709 - val_accuracy: 0.7425
Epoch 8/10
84/84 [==

### Predicting a Winner

In [6]:
# Run this cell to run the 'Input_Generator' notebook which creates functions invovled in calcualating input data
%run Input_Generator.ipynb

Using the variables `team1`, `team2`, `date`, and `home` you can enter the neccecary information about an upcomoing game! The rest of the cell uses the `model` to predict the outcome of the game with a percent certainty and prints out the result. 

In [10]:
team1 = "Shred"
team2 = "Empire"
date = "7/4/2023"
home = 1        # 1 for home game, 0 for away game, refers to team1

# Runs the function defined in Input_Generator
audl_input = get_input_data(team1,team2,date,home)

# Splits into identifying information and input data
id = audl_input[0:5]
data = audl_input[5:]

# Converts to numpy array and converts all values to floats
data = np.array(data)
data_float = data.astype(float)

# predicts outcome and stores in variable
prediction = model.predict(np.array( [data_float,]),verbose = 0)
print("AUDL AI predicts the " + id[0] + " have a " + str(round(prediction[0][0] * 100,1)) + "% chance of beating the "+ id[2] + "!")


AUDL AI predicts the Shred have a 42.3% chance of beating the Empire!
